https://www.aemo.com.au/-/media/Files/Electricity/NEM/IT-Systems-and-Change/NEMDE-queue/NEMDE_Queue_Users_Guide.pdf

https://www.accc.gov.au/system/files/Appendix%206%20-%20HoustonKemp%20-%20Investigating%20wholesale%20market%20outcomes%20-%20Met....pdf

In [1]:
import xml.etree.ElementTree as ET
import os
import pandas as pd
#path = '/Users/adam/nem-data/nemde/2018-1-1/NEMPriceSetter_2018010128800.xml'
def load_xml(path):
    #xml are hierachical data represented as tree
    tree = ET.parse(path)
    #root is highest hierachy
    root = tree.getroot()
    #here, command iterates over all children nodes of root
    data = [c.attrib for c in root]

    #print(type(data))

    df = pd.DataFrame(data)
    #print(df.shape)
    #return df
    return df.query('Market == "Energy" and RegionID == "SA1"')

In [2]:
from nemdata.nemde import form_nemde_url

In [16]:
url = form_nemde_url(2018, 1, 1).split('/')

In [17]:
fi = url[-1]

In [18]:
from nemdata.databases import Files

db = Files('nemde')

db.root

'/Users/adam/nem-data/nemde'

In [19]:
files = [fi, fi]

files = [os.path.join(db.root, f) for f in files]

In [20]:
data = []
for d in files:
    print(d)
    
    if d[-4:] != '.zip':
        
        data.append(load_xml(os.path.join(db.root, d)))
        
data = pd.concat(data)

data.index = pd.to_datetime(data.loc[:, 'PeriodID'])
data.sort_index(inplace=True)

/Users/adam/nem-data/nemde/NemPriceSetter_20180101_xml.zip
/Users/adam/nem-data/nemde/NemPriceSetter_20180101_xml.zip


ValueError: No objects to concatenate

In [ ]:
db.root